In [1]:
import numpy as np

## data i/o

In [2]:
data = 'The first appearance of an Infinity Gem occurred in 1972 in Marvel Premiere #1. It was originally called a "Soul Gem."[1][2] In 1976, a second "Soul Gem" appeared in a Captain Marvel story and established that there were six Soul Gems, each with different powers.' # should be simple plain text file

chars = list(set(data))
data_size, vocab_size = len(data),len(chars)

print('data has %d characters, %d unique.' % (data_size, vocab_size))
print('Original Text :', data, '\n')

char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

data has 263 characters, 41 unique.
Original Text : The first appearance of an Infinity Gem occurred in 1972 in Marvel Premiere #1. It was originally called a "Soul Gem."[1][2] In 1976, a second "Soul Gem" appeared in a Captain Marvel story and established that there were six Soul Gems, each with different powers. 



## HyperParameter

In [4]:
hidden_size = 5 # size of hidden layer of neurons

seq_length = 25 # number of steps to unroll the RNN for

learning_rate = 1e-1


## ModelParameter

In [14]:
Wxh = np.random.rand(hidden_size,vocab_size)*0.01
Whh = np.random.rand(hidden_size,hidden_size)*0.01
Why = np.random.rand(vocab_size,hidden_size)*0.01
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias


## Loss function

In [10]:
def lossFun(inputs,targets,hprev):
    xs,hs,ys,ps={},{},{},{}
    
    # HS[-1]에 임시저장
    hs[-1] =np.copy(hprev)
    loss = 0
    
    # forward pass
    
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1))
        
        # 이게 왜 1 일까?
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh,xs[t])+np.dot(Whh,hs[t-1])+bh)
        ys[t] = np.dot(Why,hs[t])+by
        ps[t] = np.exp(ys[t])/np.sum(np.exp(ys[t]))
        
        # [targets[t],0] 0의 의미는? 3차원 행렬이므로 ps[t][targets[t]가 리스트의 형태로 나옴 input 010 targets 102 1출력
        loss += -np.log(ps[t][targets[t],0])

    
    # backward
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh,dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        
        # 타겟의 역전파는 순정파의 -1
        dy[targets[t]] -= 1
        
        dWhy += np.dot(dy,hs[t].T)
        dby += dy
        dh = np.dot(Why.T,dy)+dhnext
        dhraw = ( 1- hs[t]*hs[t])*dh
        dbh += dhraw
        dWxh += np.dot(dhraw,xs[t].T)
        dWhh += np.dot(dhraw,hs[t-1].T)
        dhnext = np.dot(Whh.T,dhraw)
    
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        # -5,5까지 정규화
        np.clip(dparam ,-5,-5,out=dparam)
    return loss,dWxh, dWhh,dWhy,dbh,dby,hs[len(inputs)-1]



## Sample

In [11]:
def sample(h,seed_ix,n):
    x = np.zeros(((vocab_size,1)))
    x[seed_ix] = 1 
    ixes = []
    
    for t in range(n):
        
        h=np.tanh(np.dot(Wxh,x)+np.dot(Whh,h)+bh)
        y = np.dot(Why,h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.argmax(p)
        
        x= np.zeros((vocab_size,1))
        x[ix]=1
        ixes.append(ix)
    return ixes

In [16]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)

mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

smooth_loss = -np.log(1.0/vocab_size)*seq_length
    

In [24]:
while True:
    if p+seq_length+1>=len(data) or n==0:
        
        hprev = np.zeros((hidden_size,1))
        p=0
        
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    if n%100 == 0:
        
        sample_ix = sample(hprev,inputs[0],200)
        txt =''.join(ix_to_char[ix] for ix in sample_ix)
        print('-----\n %s \n --------' %(txt,))
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss* 0.999 + los*0.001
   
 

IndentationError: unindent does not match any outer indentation level (<ipython-input-24-196e38684b0d>, line 18)